In [1]:
%load_ext autoreload
%autoreload 2

from utilities import *

process_zips()
runs = extract_runs()

data = runs_to_df(runs, threshold=0.015)

Extracting runs: 100%|██████████| 20/20 [00:01<00:00, 10.15it/s]
888it [00:00, 1450366.80it/s]


In [5]:
len(runs)

18903

In [2]:
from state_analysis import extract_states_and_choices

states, choices = extract_states_and_choices(runs)

100%|██████████| 18903/18903 [00:03<00:00, 5168.49it/s]


In [95]:
import torch

n_embed = 420
batch_size = 1337
cont_features = 69
deck_size = 52

layer = torch.nn.Linear(cont_features, n_embed, bias=True)

cont = torch.rand((batch_size, 1, cont_features))

result = layer(cont)

cat = torch.rand((batch_size, deck_size, n_embed))


In [106]:
cards = set()

for run in runs:
    cards = cards.union(set(run["master_deck"]))

In [109]:
sorted(list(cards))

['A Thousand Cuts',
 'A Thousand Cuts+1',
 'Accuracy',
 'Accuracy+1',
 'Acrobatics',
 'Acrobatics+1',
 'Adaptation',
 'Adaptation+1',
 'Adrenaline',
 'Adrenaline+1',
 'After Image',
 'After Image+1',
 'Aggregate',
 'Aggregate+1',
 'All For One',
 'All For One+1',
 'All Out Attack',
 'All Out Attack+1',
 'Alpha',
 'Alpha+1',
 'Amplify',
 'Amplify+1',
 'Anger',
 'Anger+1',
 'Apotheosis',
 'Apotheosis+1',
 'Armaments',
 'Armaments+1',
 'AscendersBane',
 'Auto Shields',
 'Auto Shields+1',
 'Backflip',
 'Backflip+1',
 'Backstab',
 'Backstab+1',
 'Ball Lightning',
 'Ball Lightning+1',
 'Bandage Up',
 'Bandage Up+1',
 'Bane',
 'Bane+1',
 'Barrage',
 'Barrage+1',
 'Barricade',
 'Barricade+1',
 'Bash',
 'Bash+1',
 'Battle Trance',
 'Battle Trance+1',
 'BattleHymn',
 'BattleHymn+1',
 'Beam Cell',
 'Beam Cell+1',
 'Berserk',
 'Berserk+1',
 'Biased Cognition',
 'Biased Cognition+1',
 'Bite',
 'Bite+1',
 'Blade Dance',
 'Blade Dance+1',
 'Blasphemy',
 'Blasphemy+1',
 'Blind',
 'Blind+1',
 'Blizzard

In [6]:
from utilities import tokenize

tokenize(states[20]["relics"])

[163, 109, 90, 129, 57, 7, 177, 118, 180, 153, 26]

In [101]:
torch.cat((result, cat), dim=1).shape

torch.Size([1337, 53, 420])

In [48]:
flattened_states = list(map(flatten_state, states))

In [18]:
len(flattened_states)

28815

In [49]:
data = pd.DataFrame.from_dict(flattened_states)

In [50]:
data.head()

,character,current_hp,max_hp,gold,ascension,floor,victory,AscendersBane,Defend_R,Defend_R+1,...,Forethought+1,Impatience+1,Searing Blow+11,Good Instincts+1,Reprogram+1,Swift Strike+1,Sadistic Nature+1,Searing Blow+12,Chrysalis+1,Violence+1
0,IRONCLAD,50,80,360,13,0,True,1.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IRONCLAD,56,80,373,13,1,True,1.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IRONCLAD,60,80,388,13,2,True,1.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,IRONCLAD,65,80,405,13,4,True,1.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,IRONCLAD,63,80,435,13,5,True,1.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
removing = []

for column, value in data.sum().items():
    if type(value) is not str and value < 10:
        removing.append(column)

In [61]:
removing

['Transmutation+1',
 'Searing Blow+11',
 'Reprogram+1',
 'Sadistic Nature+1',
 'Searing Blow+12']

In [62]:
states[0]

{'character': 'IRONCLAD',
 'current_hp': 50,
 'max_hp': 80,
 'gold': 360,
 'deck': ['AscendersBane',
  'Defend_R',
  'Defend_R',
  'Defend_R',
  'Defend_R+1',
  'Shockwave+1',
  'Inflame+1',
  'Bash',
  'Strike_R',
  'Strike_R',
  'Strike_R'],
 'relics': [],
 'ascension': 13,
 'floor': 0,
 'victory': True}

In [70]:
from dataloading import SimpleDataset

dataset = SimpleDataset(states)
dataset.df.head()

,character,current_hp,max_hp,gold,ascension,floor,victory,AscendersBane,Defend_R,Defend_R+1,...,Forethought+1,Impatience+1,Searing Blow+11,Good Instincts+1,Reprogram+1,Swift Strike+1,Sadistic Nature+1,Searing Blow+12,Chrysalis+1,Violence+1
0,IRONCLAD,50,80,360,13,0,True,1.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,IRONCLAD,56,80,373,13,1,True,1.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,IRONCLAD,60,80,388,13,2,True,1.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,IRONCLAD,65,80,405,13,4,True,1.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,IRONCLAD,63,80,435,13,5,True,1.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
removing

['AscendersBane',
 'Tiny Chest',
 'Omamori',
 'Meat on the Bone',
 'Barricade',
 'Red Mask',
 'Frozen Egg 2',
 'Art of War',
 'War Paint',
 'Peace Pipe',
 'White Beast Statue',
 'Strawberry',
 'Calipers',
 'CeramicFish',
 'Molten Egg 2',
 'Bash+1',
 'Zap',
 'Dualcast',
 'Regal Pillow',
 'Toy Ornithopter',
 'MawBank',
 'Letter Opener',
 'Pantograph',
 'Auto Shields+1',
 'Whetstone',
 'Sundial',
 'Juzu Bracelet',
 'Blue Candle',
 'Du-Vu Doll',
 'Undo+1',
 'Shuriken',
 'Lantern',
 'Sweeping Beam',
 'Lockon',
 'Darkness',
 'Centennial Puzzle',
 'Toxic Egg 2',
 'Biased Cognition+1',
 'Matryoshka',
 'Akabeko',
 'Orichalcum',
 'Kunai',
 'Dark Shackles+1',
 'Happy Flower',
 'Bottled Tornado',
 'Smiling Mask',
 'MealTicket',
 'Vigilance',
 'Conclude+1',
 'BowlingBash',
 'Nunchaku',
 'Bag of Marbles',
 'Blood Vial',
 'Bronze Scales',
 'Enchiridion',
 'Question Card',
 'Ancient Tea Set',
 'Fasting2',
 'WaveOfTheHand+1',
 'Discovery+1',
 'Sanctity+1',
 'Apotheosis+1',
 'Potion Belt',
 'CurseOfTheB

In [54]:
reconstruct_state(runs[9344], 3)

{'character': 'DEFECT',
 'current_hp': 59,
 'max_hp': 95,
 'gold': 1013,
 'deck': ['AscendersBane',
  'Strike_B',
  'Defend_B',
  'Defend_B',
  'Defend_B',
  'Defend_B',
  'Zap',
  'Dualcast',
  'Strike_B',
  'Strike_B',
  'Strike_B'],
 'relics': [],
 'ascension': 13}